In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report ,confusion_matrix
import seaborn as sns
import missingno 
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import  accuracy_score
from sklearn import preprocessing
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("../input/tabletpc-priceclassification/tablet.csv")
df.dataframeName = "tablet.csv"

In [ ]:
df.head()


In [ ]:
df["MikroislemciHizi"].value_counts()
#0.5 bir yüklenme var 3.0 da düşüklük var geriye kalan veriler dengeli

In [ ]:
df["BataryaGucu"].value_counts()

In [ ]:
df["BataryaOmru"].value_counts()

In [ ]:
df["CozunurlukYükseklik"].value_counts()

In [ ]:
df["FiyatAraligi"].value_counts()

Verilerimizin  büyük oranda dengeli dağıldığını görebiliyoruz 

In [ ]:
df.info()

In [ ]:
df.corr()
#8  object türünde verimiz var onları ilerleyen zamanlarda sayısallaştırma işlemleri yapıp dahil edecez


In [ ]:
corr = df.corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values
          );


In [ ]:
sns.scatterplot(x = "CozunurlukGenislik", y = "CozunurlukYükseklik",sizes=(40, 400),hue='FiyatAraligi', data = df);

**Çözünürlük Yükseklik ve Çözünürlük Genişlik arasında Pozitif bir korelasyon vardır Ayrıca artışa göre fiyat aralığınıda görebiliyoruz **

In [ ]:
sns.scatterplot(x = "ArkaKameraMP", y = "OnKameraMP",sizes=(40, 400),hue='FiyatAraligi', data = df);

****ArkaKameraMP ve ÖnKameraMP  arasında Pozitif bir korelasyon vardır Ayrıca artışa göre fiyat aralığınıda görebiliyoruz ****

In [ ]:
sns.jointplot(x = "BataryaOmru", y = "BataryaGucu", data = df, color="blue"); 

In [ ]:
sns.swarmplot(x = 'BataryaOmru', y = 'BataryaGucu', data = df)

**BataryaGücü ile Batarya Ömrü Arasında Düzensiz bir ilişki olduğunu görüyoruz**

In [ ]:
sns.lmplot(x='MikroislemciHizi', y='RAM', fit_reg = False, hue = 'FiyatAraligi',data=df)

** Mikroişlemci Ve Ram değerleri yükseldikçe fiyatların gitgide arttığını görebiliyoruz**

In [ ]:
sns.boxplot(data = df)

In [ ]:
stats_df = df.drop(['BataryaGucu', 'MikroislemciHizi', 'OnKameraMP','ArkaKameraMP','CozunurlukYükseklik','BataryaOmru','RAM'], axis = 1)

In [ ]:
sns.boxplot(data = stats_df, notch = True, linewidth = 2.5, width = 0.50)

In [ ]:
sns.set_style('whitegrid')
sns.violinplot(x = 'Kalinlik', y = 'Agirlik', data = df)

In [ ]:
sns.distplot(df.CekirdekSayisi)

In [ ]:
sns.countplot(x='DahiliBellek', data=df)

In [ ]:
df.describe().T

**Verilerimizin Ortalama standart sapma ve medyan değerlerine baktığımızda ortalama ile 50 medyanın birbirine yakın olması verilerimizin düzgün dağıldığının göstergesidir.
Ayrıca BataryaGücü Çözünürlük Yükseklik , Genişlikte ve Ramda standart sapa çok yüksektir bunun sebebi minimum ve maksimum değerler arasında yüksek bir farkın olmasıdır. **

In [ ]:
missingno.matrix(df,figsize=(20, 10));

> **Matrisimizdeki Yatay Beyaz Cizgiler OnKameraMP ve RAM verilerinde boşluklar olduğunu Gösteriyor **

In [ ]:
df.isna( ).sum( ) 

In [ ]:
df['OnKameraMP'].mean()


In [ ]:
df['OnKameraMP']= df['OnKameraMP'].fillna(df['OnKameraMP'].mean())



In [ ]:
df['RAM']=df['RAM'].fillna(df['RAM'].mean())
# eksik veri sayısı az olduğundan dolayı ortalama değerler ile doldurmak istedim

In [ ]:
df.isnull().sum().sum()

In [ ]:
df["FiyatAraligi"].unique()

In [ ]:
pd.get_dummies(df["Renk"])

In [ ]:
df_Bluetooth=pd.get_dummies(df["Bluetooth"])
df_Bluetooth.columns = ['BluetoothVar','BluetoothYok']
df=pd.concat([df,df_Bluetooth],axis=1)

In [ ]:
df_4G=pd.get_dummies(df["4G"])
df_4G.columns = ['4GVar','4GYok']
df=pd.concat([df,df_4G],axis=1)

In [ ]:
df_3G=pd.get_dummies(df["3G"])
df_3G.columns = ['3GVar','3GYok']
df=pd.concat([df,df_3G],axis=1)

In [ ]:
df_CiftHat=pd.get_dummies(df["CiftHat"])
df_CiftHat.columns = ['CiftHatVar','CiftHatYok']
df=pd.concat([df,df_CiftHat],axis=1)

In [ ]:
df_Dokunmatik=pd.get_dummies(df["Dokunmatik"])
df_Dokunmatik.columns = ['DokunmatikVar','DokunmatikYok']
df=pd.concat([df,df_Dokunmatik],axis=1)

In [ ]:
df_WiFi=pd.get_dummies(df["WiFi"])
df_WiFi.columns = ['WiFiVar','WiFiYok']
df=pd.concat([df,df_WiFi],axis=1)

In [ ]:
df.drop(['WiFi','Bluetooth','3G','4G','CiftHat','Dokunmatik'], axis=1, inplace=True)

In [ ]:
label_encoder = preprocessing.LabelEncoder()

In [ ]:
df['Renk'] = label_encoder.fit_transform(df['Renk'])

In [ ]:
df.drop(["WiFiYok","BluetoothYok","3GYok","4GYok","CiftHatYok","DokunmatikYok"], axis=1, inplace=True)

**Kategorik Değişkenleri Sayısallaştırıdğımız için bir daha bakalım Korelasyon Matrislerimize**

In [ ]:
df.corr()

In [ ]:
corr = df.corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values
          );


In [ ]:
sns.scatterplot(x = "4GVar", y = "3GVar", hue = "FiyatAraligi", data = df);

In [ ]:
sns.barplot(x="FiyatAraligi",y="RAM",data=df)

    **2.Aşama Veri Ön işleme**

**1.Aşamada Eksik verileri Bulup Sayıları Az olduğundan Ortalama Değerler ile Doldurmuştuk**

In [ ]:
X = df.drop("FiyatAraligi",axis=1)
y = df["FiyatAraligi"]

In [ ]:
X

In [ ]:
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=0)

In [ ]:
df.info()

In [ ]:
y_test

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
y_train

**Eğitim Kümemizi oluşturuyoruz ve Tek Tek inceliyoruz**

**  GaussianNB Modeli **

In [ ]:
gnb = GaussianNB()

In [ ]:
modelgnb=gnb.fit(X,y)

In [ ]:
y_pred = modelgnb.predict(X_test)
y_pred

In [ ]:
accuracy_score(y_test,y_pred)

** 0.796 oranında doğru tahminlerde bulunulduğunu gösteriyor**

In [ ]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred)
karmasiklik_matrisi

In [ ]:
F1Score = f1_score(y_test, y_pred, average = 'weighted')  
F1Score

In [ ]:
RecallScore = recall_score(y_test, y_pred, average='weighted')
RecallScore

In [ ]:
PrecisionScore = precision_score(y_test, y_pred, average='weighted')
PrecisionScore

In [ ]:
print(classification_report(y_test, y_pred)) 

In [ ]:
cross_val_score(modelgnb, X, y, cv = 7)

** Precision recal ve f1 değerlerimiz  0.80 civarında ve 1 e yakın dolayısıyla başarılı eğitimimiz başarılı olmuştur **

**DecisionTree**

In [ ]:
tree = DecisionTreeClassifier()
modeltree = tree.fit(X_train,y_train)

In [ ]:
modeltree

In [ ]:
y_pred = modeltree.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred)
print(karmasiklik_matrisi)

In [ ]:
F1Score = f1_score(y_test, y_pred, average = 'weighted')  


In [ ]:
RecallScore = recall_score(y_test, y_pred, average='weighted')

In [ ]:
PrecisionScore = precision_score(y_test, y_pred, average='weighted')

In [ ]:
print(classification_report(y_test, y_pred)) 

** Precision recal ve f1 değerlerimiz  0.83 civarında ve 1 e yakın dolayısıyla başarılı eğitimimiz başarılı olmuştur **

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
tree = DecisionTreeClassifier(criterion='entropy')
modeltree = tree.fit(X_train, y_train)

In [ ]:
y_pred1 = modeltree.predict(X_test)

In [ ]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred1)
print(karmasiklik_matrisi)

In [ ]:
PrecisionScore = precision_score(y_test, y_pred1, average='weighted')

In [ ]:
RecallScore = recall_score(y_test, y_pred1, average='weighted')

In [ ]:
F1Score = f1_score(y_test, y_pred1, average = 'weighted')  

In [ ]:
print(classification_report(y_test, y_pred1))

In [ ]:
accuracy_score(y_test, y_pred1)

** default olarak gelen  (criterion = “gini”) yerine  (criterion = “entropy”) kullanıldığında başarının artacağı Söyleyen Ekip Lideri Yanılmış olmalı çünkü 0.01 oranında azalma görülmüştür **

**KNN Modeli **

In [ ]:
knn = KNeighborsClassifier()
knn_model = knn.fit(X_train, y_train)

In [ ]:
y_pred2 = knn_model.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred2)

In [ ]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred1)
print(karmasiklik_matrisi)

In [ ]:
F1Score = f1_score(y_test, y_pred2, average = 'weighted')  
F1Score

In [ ]:
RecallScore = recall_score(y_test, y_pred2, average='weighted')
RecallScore

In [ ]:
PrecisionScore = precision_score(y_test, y_pred2, average='weighted')
PrecisionScore

In [ ]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred)
print(classification_report(y_test, y_pred2))

In [ ]:
KomsuSayisi = [] 

for k in range(2,15,1): 
    knnKomsuSayisi = KNeighborsClassifier(n_neighbors = k)
    knnKomsuSayisi.fit(X_train,y_train)
    KomsuSayisi.append(knnKomsuSayisi.score(X_test, y_test))

plt.plot(range(2,15,1),KomsuSayisi)
plt.xlabel("En Yakın Komşu Sayısı =K")
plt.ylabel("Doğruluk Oranı")
plt.show()

In [ ]:
knn_Komsu = KNeighborsClassifier(n_neighbors =13)
knn_Komsu = knn_Komsu.fit(X_train, y_train)

In [ ]:
cross_val_score(knn_model, X_test, y_test, cv = 5)

In [ ]:
PrecisionScore = precision_score(y_test, y_pred, average='weighted')
PrecisionScore

In [ ]:
PrecisionScore = precision_score(y_test, y_pred, average='weighted')
PrecisionScore

In [ ]:
PrecisionScore = precision_score(y_test, y_pred, average='weighted')
PrecisionScore

In [ ]:
print(classification_report(y_test, y_pred))

**Değerlerimiz yine yüksek ve birbirine yakın **